# Segmenting and Clustering Toronto Assignment
This is part 3 of the assignment

<b>Inserting substantive code from first two parts of the assignment.</b>

In [1]:
import pandas as pd
import numpy as np
import requests
!pip install lxml
!pip install html5lib
!pip install beautifulsoup4
!pip install folium
import folium
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print ('Libraries imported.')
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url)
Toronto_df=df[0]
Toronto_df.drop(Toronto_df[Toronto_df.Borough == 'Not assigned'].index, inplace=True)
Toronto_df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)
Toronto_df2 = Toronto_df.groupby(['Postal Code', 'Borough'], sort=False)['Neighborhood'].apply(','.join).reset_index()
geo_coord_df = pd.read_csv('https://cocl.us/Geospatial_data', header=0)
geo_coord_df.sort_values(by='Postal Code', ascending=True)
Toronto_with_geo_coord_df = pd.merge(Toronto_df, geo_coord_df, on='Postal Code')
Toronto_with_geo_coord_df.rename(columns={'Postal Code' : 'PostalCode'}, inplace=True)
Toronto_with_geo_coord_df.shape

Libraries imported.


(103, 5)

In [2]:
Toronto_with_geo_coord_df.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [3]:
address='Toronto, Ontario'
geolocator=Nominatim(user_agent="Toronto_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print("The geographical coordinates of Toronto are {}, {},".format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817, -79.3839347,


In [4]:
map_Toronto=folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(Toronto_with_geo_coord_df['Latitude'], Toronto_with_geo_coord_df['Longitude'], Toronto_with_geo_coord_df['Borough'], Toronto_with_geo_coord_df['Neighborhood']):
    label='{}, []'.format(neighborhood, borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)
map_Toronto

<b>Define Foursquare Credentials and Version</b>

In [5]:
CLIENT_ID = 'CCV0WH2F0XZIXGEGK40O2410LU2CVG013GU3RATKSJJGRKJJ' # my Foursquare ID
CLIENT_SECRET = '14XTBAHQNJBN35UNTAZP2RBS12PS2KMQZK54BUWWDL1LFE0B' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CCV0WH2F0XZIXGEGK40O2410LU2CVG013GU3RATKSJJGRKJJ
CLIENT_SECRET:14XTBAHQNJBN35UNTAZP2RBS12PS2KMQZK54BUWWDL1LFE0B


<b>Per assignment instructions, limiting scope to only the boroughs with 'Toronto' in the name</b>

In [6]:
Toronto_data = Toronto_with_geo_coord_df[Toronto_with_geo_coord_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data.sort_values(by='Borough')
Toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [7]:
print("Shape of Toronto only dataframe: ",Toronto_data.shape)

Shape of Toronto only dataframe:  (39, 5)


<b>Locate the first neighborhood in the dataframe, and retrieve the latitude and longitude values.</b>

In [8]:
Toronto_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [9]:
neighborhood_latitude=Toronto_data.loc[0, 'Latitude']
neighborhood_longitude=Toronto_data.loc[0, 'Longitude']
neighborhood_name=Toronto_data.loc[0, 'Neighborhood']
print('Latitude and Longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and Longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Getting Top 100 Venues that are in Regent Park, Harbourfront within a radius of 500 meters.

In [10]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=CCV0WH2F0XZIXGEGK40O2410LU2CVG013GU3RATKSJJGRKJJ&client_secret=14XTBAHQNJBN35UNTAZP2RBS12PS2KMQZK54BUWWDL1LFE0B&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [11]:
#sending GET request to examine the results
results=requests.get(url).json
results

<bound method Response.json of <Response [200]>>

<b>Exploring the Neighborhoods</b>

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [ ]:
Toronto_venues=getNearbyVenues(names=Toronto_data['Neighborhood'], latitudes=Toronto_data['Latitude'], longitudes=Toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government


In [ ]:
print(Toronto_venues.shape)
Toronto_venues.head()

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

In [ ]:
print('{} venues were returned by Foursquare.'.format(Toronto_venues.shape[0]))

In [ ]:
Toronto_venues.groupby('Neighborhood').count()

<b>Conduct Analysis of Each Neighborhood</b>

In [ ]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood']
# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
Toronto_onehot.head()

In [ ]:
Toronto_onehot.shape

In [ ]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

In [ ]:
Toronto_grouped.shape

In [ ]:
num_top_venues = 5
for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

<b>Putting it into a pandas dataframe.</b>
</n> 
First, I'll write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)  
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# creating columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# creating a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']
for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

<b>Running K-Means to Cluster the Neighborhoods into 5 Clusters.</b>

In [ ]:
# set number of clusters
kclusters = 5
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_merged = Toronto_data
# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged.head() 

<b>Finally, let's visualize the resulting clusters</b>

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1],fill=True, fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
map_clusters


# Examining the Clusters

I will examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, I will then assign a name to each cluster.

Cluster 1: Restaurants, Coffee Shops and Shopping

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Cluster 2: Restaurants, Tennis and Entertainment

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Cluster 3: Recreational

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

Cluster 4: Restaurants and Recreation

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]